## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_vacation.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Kapaa,22.0752,-159.3190,59.00,82,13,6.91,US,few clouds
1,1,Sitka,57.0531,-135.3300,35.60,93,90,4.61,US,overcast clouds
2,2,Mataura,-46.1927,168.8643,49.21,94,3,1.16,NZ,clear sky
3,3,Aklavik,68.2191,-135.0107,-27.40,68,20,1.57,CA,few clouds
4,4,Ushuaia,-54.8000,-68.3000,51.80,50,40,19.57,AR,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
8,8,Wewak,-3.5534,143.6268,79.12,82,100,5.68,PG,overcast clouds
10,10,Bengkulu,-3.8004,102.2655,76.86,85,78,6.55,ID,broken clouds
12,12,Sao Felix Do Xingu,-6.6447,-51.9950,79.63,78,100,1.72,BR,overcast clouds
13,13,Luderitz,-26.6481,15.1594,77.23,46,0,30.27,NaN,clear sky
19,19,Rikitea,-23.1203,-134.9692,77.47,74,100,13.20,PF,overcast clouds
20,20,Cape Town,-33.9258,18.4232,78.01,65,0,18.41,ZA,clear sky
25,25,Vaini,-21.2000,-175.2000,77.00,100,75,4.61,TO,broken clouds
27,27,Castro,-24.7911,-50.0119,76.21,51,0,5.57,BR,clear sky
31,31,Hithadhoo,-0.6000,73.0833,81.99,76,100,14.25,MV,overcast clouds
32,32,Tual,-5.6667,132.7500,83.91,75,100,22.95,ID,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                3
Current Description    0
dtype: int64

In [8]:
preferred_cities_df.count()

City_ID                234
City                   234
Lat                    234
Lng                    234
Max Temp               234
Humidity               234
Cloudiness             234
Wind Speed             234
Country                231
Current Description    234
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Wewak,PG,79.12,overcast clouds,-3.5534,143.6268,
10,Bengkulu,ID,76.86,broken clouds,-3.8004,102.2655,
12,Sao Felix Do Xingu,BR,79.63,overcast clouds,-6.6447,-51.9950,
19,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,
20,Cape Town,ZA,78.01,clear sky,-33.9258,18.4232,
25,Vaini,TO,77.00,broken clouds,-21.2000,-175.2000,
27,Castro,BR,76.21,clear sky,-24.7911,-50.0119,
31,Hithadhoo,MV,81.99,overcast clouds,-0.6000,73.0833,
32,Tual,ID,83.91,overcast clouds,-5.6667,132.7500,
41,Faanui,PF,81.37,clear sky,-16.4833,-151.7500,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Wewak,PG,79.12,overcast clouds,-3.5534,143.6268,In Wewak Boutique Hotel
10,Bengkulu,ID,76.86,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
12,Sao Felix Do Xingu,BR,79.63,overcast clouds,-6.6447,-51.9950,Rio Xingu Camping
19,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,Pension Maro'i
20,Cape Town,ZA,78.01,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"


In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()



City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [15]:
clean_hotel_df = hotel_df.dropna()

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "cities_1.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
print(hotel_info)

['\n<dl>\n<dt>City</dt><dd>Wewak</dd>\n<dt>Country</dt><dd>PG</dd>\n<dt>Current Description</dt><dd>overcast clouds</dd>\n<dt>Max Temp</dt><dd>79.12 °F</dd>\n</dl>\n', '\n<dl>\n<dt>City</dt><dd>Bengkulu</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Description</dt><dd>broken clouds</dd>\n<dt>Max Temp</dt><dd>76.86 °F</dd>\n</dl>\n', '\n<dl>\n<dt>City</dt><dd>Sao Felix Do Xingu</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Description</dt><dd>overcast clouds</dd>\n<dt>Max Temp</dt><dd>79.63 °F</dd>\n</dl>\n', '\n<dl>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Description</dt><dd>overcast clouds</dd>\n<dt>Max Temp</dt><dd>77.47 °F</dd>\n</dl>\n', '\n<dl>\n<dt>City</dt><dd>Cape Town</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Description</dt><dd>clear sky</dd>\n<dt>Max Temp</dt><dd>78.01 °F</dd>\n</dl>\n', '\n<dl>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>TO</dd>\n<dt>Current Description</dt><dd>broken clouds</dd>\n<dt>Max Temp</dt><dd>77.0 °F</d

In [19]:
locations.head()

,Lat,Lng
8,-3.5534,143.6268
10,-3.8004,102.2655
12,-6.6447,-51.9950
19,-23.1203,-134.9692
20,-33.9258,18.4232


In [20]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))